# Web Scraping of MyBidMatch Entries Using BeautifulSoup
### David Lowe
### May 15, 2020

## Loading Libraries and Packages

In [1]:
# !pip install python-dotenv

In [2]:
# Retrieve CPU information from the system
# ncpu = !nproc
# print("The number of available CPUs is:", ncpu[0])

In [1]:
import pandas as pd
import os
import smtplib
import sys
import pymysql
import requests
from requests.exceptions import HTTPError
from requests.exceptions import ConnectionError
from email.message import EmailMessage
from datetime import datetime
from datetime import date
from random import randint
from time import sleep
from bs4 import BeautifulSoup

In [2]:
# Mount Google Drive locally for accessing files
# from google.colab import drive
# drive.mount('/content/gdrive')
# env_path = '/content/gdrive/My Drive/Colab Notebooks/'

env_path = "./"

In [3]:
startTimeScript = datetime.now()

## Setting up the basic functions

In [4]:
# Define the function for sending the status notification emails
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if (sender is None) or (receiver is None) or (gateway is None) or (smtpuser is None) or (password is None):
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from myBidMatch Python Web Scraping Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [5]:
# Define the function for storing the scraped records
def storeDB(posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text):
    print("Inserting record:", posting_date, '|', source_tag, '|', agency_name, '|', fsg_tag, '|', article_title, '|', search_keywords, '|', notice_heading, '|', department_url, '|', notice_url)
    try:
        cur.execute("INSERT INTO bsoup_mybidmatch_notices (posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text))
        cur.connection.commit()
        print("Successfully inserted the record into the database.")
    except:
        print("Failed to insert the record into the database.")

## Setting up the necessary parameters

In [6]:
# Set up the verbose flag to print detailed messages for debugging (setting True will activate!)
verbose = False

# Set up the sendNotification flag to send progress emails (setting True will send emails!)
notifyStatus = False

# Set up the writeToDB flag to write records into the database (setting True will record!)
writeToDB = True

# Set up the writeJSON flag to write records into a JSON document (setting True will record!)
writeJSON = False

# Set up target date to collect the article from that date only
targetDate = date(2020, 5, 15)
if targetDate is None:
    processAll = True
else: processAll = False

In [7]:
if notifyStatus: email_notify("The web scraping process for myBidMatch has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [8]:
# Specifying the URL of desired web page to be scrapped
websiteURL = "http://www.mybidmatch.com"
startingURL = websiteURL + "/go?sub=55AB9731-0E3E-4BC0-B0E3-56EF81DA7FD4"

# Creating an html document from the URL
uastring = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:76.0) Gecko/20100101 Firefox/76.0"
headers={'User-Agent': uastring}

In [9]:
if writeToDB:
    # Set up the database connection strings and environment
    from dotenv import load_dotenv
    env_path = env_path + "database.env"
    load_dotenv(dotenv_path=env_path)
    db_host = os.environ.get('DB_HOST')
    db_user = os.environ.get('DB_USER')
    db_pass = os.environ.get('DB_PASS')
    db_name = os.environ.get('DB_NAME1')
    print("Trying to open a connection to host", db_host, "as user", db_user, "for database", db_name)

    # Connect to the database
    try:
        conn = pymysql.connect(host=db_host, user=db_user, password=db_pass, db=db_name, charset='utf8')
        cur = conn.cursor()
        cur.execute("USE %s" % (db_name))
        print("Successfully opened a connection to host", db_host, "as user", db_user, "for database", db_name)
    except:
        print("Unable to open a connection to host", db_host, "as user", db_user, "for database", db_name)
        writeToDB = False

Trying to open a connection to host ec2-35-170-217-54.compute-1.amazonaws.com as user scrapinguser for database webscraping
Successfully opened a connection to host ec2-35-170-217-54.compute-1.amazonaws.com as user scrapinguser for database webscraping


## Performing the Scraping and Processing

In [10]:
if notifyStatus: email_notify("The page loading and item extraction process has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [11]:
try:
    s = requests.Session()
    resp = s.get(startingURL, headers=headers)
    if (verbose): print(resp.text)
except HTTPError as e:
    print('The server could not serve up the web page!')
    sys.exit("Script processing cannot continue!!!")
except ConnectionError as e:
    print('The server could not be reached due to connection issues!')
    sys.exit("Script processing cannot continue!!!")

if (resp.status_code==requests.codes.ok):
    print('Successfully accessed the company web page: ' + startingURL)
    searchPage = BeautifulSoup(resp.text, 'lxml')
    if verbose: print(searchPage)

Successfully accessed the company web page: http://www.mybidmatch.com/go?sub=55AB9731-0E3E-4BC0-B0E3-56EF81DA7FD4


In [12]:
# Setting up a dataframe to capture the records
df = pd.DataFrame(columns=['Posting_Date', 'Source_Tag', 'Agency_Name', 'FSG_Tag', 'Article_Title', 'Search_Keywords', 'Notice_Heading', 'Department_URL', 'Notice_URL', '''Notice_Text'''])
i = 0

In [13]:
done = False

search_listing = searchPage.find("table", class_="data").find_all("tr")
if verbose: print(search_listing)

In [14]:
for search_item in search_listing :
    search_element = search_item.find_all("td")
    posting_date_text = search_element[0].string
    posting_date = datetime.strptime(posting_date_text, '%A, %b %d, %Y').date()
    number_articles = int(search_element[1].string)
    group_url = websiteURL + search_item.find('a').get('href')

    if (number_articles > 0) and ((posting_date == targetDate) or processAll):
        # Adding random wait time so we do not hammer the website needlessly
        waitTime = randint(3,6)
        print("Waiting " + str(waitTime) + " seconds to process next article grouping page...")
        sleep(waitTime)
        
        try:
            s = requests.Session()
            resp = s.get(group_url, headers=headers)
            if (verbose): print(resp.text)
        except HTTPError as e:
            print('The server could not serve up the web page!')
            sys.exit("Script processing cannot continue!!!")
        except ConnectionError as e:
            print('The server could not be reached due to connection issues!')
            sys.exit("Script processing cannot continue!!!")
        if (resp.status_code==requests.codes.ok):
            print('Successfully accessed the article grouping web page: ' + group_url)
            noticePage = BeautifulSoup(resp.text, 'lxml')
                
        notice_listing = noticePage.find("table", class_="data").find_all("tr")
        if verbose: print(notice_listing)

        for notice_item in notice_listing :
            notice_element = notice_item.find_all("td")
            source_tag = notice_element[1].string.strip()
            agency_name = notice_element[2].string.strip()
            if (notice_element[3].string is None):
                fsg_tag = "N/A"
            else:
                fsg_tag = notice_element[3].string.strip()
            article_title = notice_element[4].string.strip()
            search_keywords = notice_element[5].string.strip()
            notice_url = websiteURL + notice_item.find('a').get('href')

            # Adding random wait time so we do not hammer the website needlessly
            waitTime = randint(2,5)
            print("Waiting " + str(waitTime) + " seconds to process next notice page...")
            sleep(waitTime)

            try:
                s = requests.Session()
                resp = s.get(notice_url, headers=headers)
                if (verbose): print(resp.text)
            except HTTPError as e:
                print('The server could not serve up the web page!')
                sys.exit("Script processing cannot continue!!!")
            except ConnectionError as e:
                print('The server could not be reached due to connection issues!')
                sys.exit("Script processing cannot continue!!!")

            if (resp.status_code==requests.codes.ok):
                print('Successfully accessed the notice web page: ' + notice_url)
                detailPage = BeautifulSoup(resp.text, 'lxml')
                notice_heading = detailPage.find("h4").string
                if (notice_heading is None): notice_heading = detailPage.find("h4").contents[0]
                notice_text = detailPage.find("div", class_="art-box").prettify()
                links_in_detail = detailPage.find("div", class_="art-box").find_all('a')
                if len(links_in_detail) > 0 : department_url = links_in_detail[-1].get('href')
                else: department_url = None

            if verbose: print(posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text)
            df.loc[i] = [posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text]
            if writeToDB: storeDB(posting_date, source_tag, agency_name, fsg_tag, article_title, search_keywords, notice_heading, department_url, notice_url, notice_text)
            else: print("Found record:", posting_date, '|', source_tag, '|', agency_name, '|', fsg_tag, '|', article_title, '|', search_keywords, '|', notice_heading, '|', department_url, '|', notice_url)
            i = i + 1

Waiting 4 seconds to process next article grouping page...
Successfully accessed the article grouping web page: http://www.mybidmatch.com/go?doc=DF84EE4E-A77C-496C-82F1-38EF4330B2ED
Waiting 5 seconds to process next notice page...
Successfully accessed the notice web page: http://www.mybidmatch.com/article?doc=DF84EE4E-A77C-496C-82F1-38EF4330B2ED&seq=1
Inserting record: 2020-05-15 | procure | HOMELAND SECURITY | D | Appointment Scheduling Software | naics!541511; software; | HOMELAND SECURITY, DEPARTMENT OF, US COAST GUARD, HQ CONTRACT OPERATIONS (CG-912)(000, WASHINGTON            DC 20593,  WASHINGTON DC 20593  | https://beta.sam.gov/opp/12b010d71bda461897e8a5c5432ec159/view? | http://www.mybidmatch.com/article?doc=DF84EE4E-A77C-496C-82F1-38EF4330B2ED&seq=1
Successfully inserted the record into the database.
Waiting 2 seconds to process next notice page...
Successfully accessed the notice web page: http://www.mybidmatch.com/article?doc=DF84EE4E-A77C-496C-82F1-38EF4330B2ED&seq=2
Inser

In [15]:
print('Finished finding all available articles on the web pages!')
print('Number of article processed:', i)

Finished finding all available articles on the web pages!
Number of article processed: 80


In [16]:
if writeToDB:
    try:
        cur.close()
        conn.close()
        print("Successfully closed the connection to host", db_host, "as user", db_user, "for database", db_name)
    except:
        print("Unable to close the connection to host", db_host, "as user", db_user, "for database", db_name)

Successfully closed the connection to host ec2-35-170-217-54.compute-1.amazonaws.com as user scrapinguser for database webscraping


## Organizing Data and Producing Outputs

In [106]:
if writeJSON:
    out_file = df.to_json(orient='records')
    with open('web-scraping-py-bsoup-mybidmatch.json', 'w') as f:
        f.write(out_file)
    print('Total number of records written to file:', len(df))

In [107]:
if notifyStatus: email_notify("The web scraping process for myBidMatch has completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [108]:
print ('Total time for the script:', (datetime.now() - startTimeScript))

Total time for the script: 0:04:14.616541
